In [4]:

import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define the paths to the images and CSV files
base_dir = '/home/mahin/Downloads/retinanet'
train_csv_path = os.path.join(base_dir, 'train', '_annotations.csv')
test_csv_path = os.path.join(base_dir, 'test', '_annotations.csv')
val_csv_path = os.path.join(base_dir, 'valid', '_annotations.csv')

# Function to read the dataset from CSV files
def load_dataset(csv_path, images_dir):
    df = pd.read_csv(csv_path)
    images = []
    labels = []

    for _, row in df.iterrows():
        # Get image file path
        image_file = row.iloc[0]  # Filename is in the first column
        image_path = os.path.join(images_dir, image_file)

        # Read and preprocess the image
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            image = image / 255.0  # Normalize
            images.append(image)

            # Get the class label
            class_label = row.iloc[4]  # Class label is in the fifth column
            labels.append(class_label)

    # Convert lists to arrays
    images = np.array(images)
    labels = np.array(labels)

    return np.array(images), np.array(labels)

# Load the datasets
train_images, train_labels = load_dataset(train_csv_path, os.path.join(base_dir, 'train'))
test_images, test_labels = load_dataset(test_csv_path, os.path.join(base_dir, 'test'))
val_images, val_labels = load_dataset(val_csv_path, os.path.join(base_dir, 'valid'))

# Convert labels to one-hot encoding
num_classes = len(set(train_labels))  # Update this if needed
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes)

# Set up ImageDataGenerator for augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ImageDataGenerators for the train, validation, and test sets
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow(val_images, val_labels, batch_size=32)
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)

# Now, you can use these generators to train a model using VGG16 architecture or any other model.


IndexError: index 537 is out of bounds for axis 1 with size 292